In [1]:
import pandas as pd
import numpy as np
import soundfile as sf
import librosa
from IPython.display import Audio
import os
from scipy.io.wavfile import write as write_wav
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

### spotify playlists

* [jamaican patois](https://open.spotify.com/playlist/54AaUsgTHhmsdfhT5lMMdl)
* [ghanaian pidgin](https://open.spotify.com/playlist/74Nm56ijqC9O9Z2qF3kC5X)
* [american english](https://open.spotify.com/playlist/4WjFFH3mIMD6EEpDmPZKc3)
* [european spanish](https://open.spotify.com/playlist/38hpr6Usrjtey7IkdNHlaN)
* [chinese mandarin](https://open.spotify.com/playlist/76XEwiwfRRtg3QDeB2ttCD)
* [indian hindu](https://open.spotify.com/playlist/4m9VYOmySJiM7wv14um6lZ)

In [2]:
# create a class object to store song information
class Song:
    
    def __init__(self, name, audio_vector, lang, samplerate, length):
        self.name = name
        self.audio = audio_vector     
        self.lang = lang              
        self.samplerate = samplerate
        self.length = length

# load data 
* save values into Song objects (language + sample rate)
* load audio into 1D vectors

In [3]:
def process_song(path, lang, sr=22050, clip_len=30):
    
    # load song with the same samplerate as patois songs
    y, _ = librosa.load(path, sr=sr, mono=True)
    name = os.path.splitext(os.path.basename(path))[0].lower()

    # total duration in seconds
    duration = librosa.get_duration(y=y, sr=sr)
    center = duration / 2

    # find center 2-minute segment
    start = int((center - 120 / 2) * sr)
    end = int((center + 120 / 2) * sr)
    y = y[start:end]

    # split into 15-second clips (8 clips from 2-minute segment)
    clip_size = clip_len * sr
    clips = [y[i:i + clip_size] for i in range(0, len(y), clip_size) if len(y[i:i + clip_size]) == clip_size]
    
    # wrap into Song objects with appropriate split assignments
    return [Song(name, clip, lang, sr, clip_len) for clip in clips]


In [4]:
# create list of Song objects for all languages
languages = ['patois', 'mandarin', 'english', 'spanish', 'hindi', 'pidgin']
mp3_files = []

total_files = sum(len([f for f in os.listdir(f'./{lang}') if f.endswith('.mp3')]) for lang in languages)
processed = 0

for lang in languages:
    folder = f'./{lang}'
    for file in os.listdir(folder):
        if file.endswith('.mp3'):
            path = os.path.join(folder, file)
            try:
                clips = process_song(path, lang)
                mp3_files.extend(clips)
                processed += 1
                if processed % 50 == 0:
                    print(f"Processed {processed}/{total_files} files...")
            except Exception as e:
                print(f"Error processing {file}: {e}")

print(f"Completed! Processed {processed} files, created {len(mp3_files)} clips total.")

Processed 50/575 files...
Processed 100/575 files...
Processed 150/575 files...
Processed 200/575 files...
Processed 250/575 files...
Processed 300/575 files...


[src/libmpg123/layer3.c:INT123_do_layer3():1844] error: dequantization failed!
Note: Illegal Audio-MPEG-Header 0x77bd7d56 at offset 945978.
Note: Trying to resync...
Note: Skipped 134 bytes in input.
Note: Illegal Audio-MPEG-Header 0x0e9b81e3 at offset 1082816.
Note: Trying to resync...
Note: Skipped 98 bytes in input.
Note: Illegal Audio-MPEG-Header 0xd30bc33e at offset 3202978.
Note: Trying to resync...
Note: Skipped 152 bytes in input.


Processed 350/575 files...
Processed 400/575 files...
Processed 450/575 files...
Processed 500/575 files...
Processed 550/575 files...
Completed! Processed 575 files, created 2270 clips total.


# visualize and listen to song audios

In [5]:
# include UI to listen to songs before and after sampling to ensure there is no data corruption
def visualize_audio(audio_vector, sampling_rate=22050, seconds=None, out_dir=None):
    """
    purpose:
        visualize your 1D audio vector as playable audio
    params:
        audio_vector: 1D numpy array of raw audio data
        sampling_rate: sample rate of your audio
        seconds: number of seconds to play
        out_dir: directory to save WAV file (None = just display)
    output:
        None if just display, audio saved location if input provodied for out_dir
    """
    
    # play only a portion
    if seconds is not None:
        samples_to_play = seconds * sampling_rate
        waveform = audio_vector[:samples_to_play]
    else:
        waveform = audio_vector
    
    # normalize to prevent clipping
    waveform = waveform / np.max(np.abs(waveform))
    
    # display the audio if no path is provided
    if out_dir is None:
        return Audio(waveform, rate=sampling_rate)
    
    # save the audio to a file
    os.makedirs(out_dir, exist_ok=True)
    audio_path = os.path.join(out_dir, "generated_audio.wav")
    
    # convert to 16-bit PCM for WAV file
    waveform_int16 = (waveform * 32767).astype(np.int16)
    write_wav(audio_path, sampling_rate, waveform_int16)
    return audio_path


In [6]:
# listen to the first song in the dataset
song = mp3_files[0]
sample = song.audio
visualize_audio(sample)

In [7]:
# listen to a downsample version of the first song in the dataset
downsample = librosa.resample(y=sample, orig_sr=22050, target_sr=12000)
visualize_audio(downsample, 12000)

In [8]:
def build_dataset(files):

    # intialize variables
    X, y = [], []
    curr_sr, down_sr = 22050, 12000
    hop_size = 256
    dft = 512
    mel_bins = 96

    
    for track in files:

        # downmix and downsample the signals to 12 kHz 
        downsample = librosa.resample(y=track.audio, orig_sr=curr_sr, target_sr=down_sr)
        
        # extract Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=downsample, sr=down_sr, hop_length = hop_size, 
            n_fft = dft, n_mels=mel_bins
        )

        # convert to log scale (dB)
        log_mel = librosa.power_to_db(mel_spec, ref=np.max)
        
        # add transposed matrix since RNN expect 3D input: samples, timesteps, features
        X.append(log_mel.T)
        y.append(track.lang)
    
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    # encode targets
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    y_onehot = to_categorical(y_encoded)
    labels_inorder = list(encoder.classes_)
    
    print(f"Dataset shape: {X.shape}")
    print(f"Labels (In Order): {labels_inorder}")
    
    # split into train, validation and test sets
    x_train, x_eval, y_train, y_eval = train_test_split(
        X, y_onehot, test_size=0.4, random_state=42, stratify=y_encoded
    )
    y_eval_encoded = np.argmax(y_eval, axis=1)
    x_val, x_test, y_val, y_test = train_test_split(
        x_eval, y_eval, test_size=0.5, random_state=42, stratify=y_eval_encoded
    )
    
    # store preprocessed data
    data = {
        'labels_inorder': labels_inorder,
        'train_features': x_train, 'train_labels': y_train,
        'val_features': x_val, 'val_labels': y_val,
        'test_features': x_test, 'test_labels': y_test
    }
    
    print(f"Train set: {x_train.shape}")
    print(f"Validation set: {x_val.shape}")
    print(f"Test set: {x_test.shape}")
    
    return data

In [9]:
data = build_dataset(mp3_files)

Dataset shape: (2270, 1407, 96)
Labels (In Order): ['english', 'hindi', 'mandarin', 'patois', 'pidgin', 'spanish']
Train set: (1362, 1407, 96)
Validation set: (454, 1407, 96)
Test set: (454, 1407, 96)


In [10]:
# save data as a file to load for analysis
np.savez_compressed(
    "spotify_dataset.npz",
    labels_inorder=data['labels_inorder'],
    train_features=data["train_features"],
    train_labels=data["train_labels"],
    val_features=data["val_features"],
    val_labels=data["val_labels"],
    test_features=data["test_features"],
    test_labels=data["test_labels"]
)